<a href="https://colab.research.google.com/github/AndreaCamilloni/BigDataInMediaTechnology/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [68]:
import pandas as pd
from bs4 import BeautifulSoup

In [69]:
path = "/content/drive/MyDrive/Group2/Lab1/"

train_data = pd.read_csv(path + 'train.csv')
test_data = pd.read_csv(path + 'test.csv')
valid_data = pd.read_csv(path + 'evaluation.csv')

train_data.size, test_data.size, valid_data.size

(15000, 5000, 10000)

In [70]:
pd.set_option('display.max_columns', None) #for display purpose

# Pre-processing


In [71]:
import re
import emoji

In [72]:
def extract_emojis(text):
  return ','.join(c for c in text if c in emoji.EMOJI_DATA)

def extract_hash_tags(s):
    return ','.join(part[1:] for part in s.split() if part.startswith('#'))


train_data['emoji']=train_data['text'].apply(lambda f : extract_emojis(f))
train_data['hashtags']=train_data['text'].apply(lambda f : extract_hash_tags(f))

test_data['emoji']=test_data['text'].apply(lambda f : extract_emojis(f))
test_data['hashtags']=test_data['text'].apply(lambda f : extract_hash_tags(f))

valid_data['emoji']=valid_data['text'].apply(lambda f : extract_emojis(f))
valid_data['hashtags']=valid_data['text'].apply(lambda f : extract_hash_tags(f))


In [73]:

def preprocess(string):
    #remove html tags
    CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    result = re.sub(CLEANR,'',string)
    #remove newline 
    result = re.sub(r'\\t|\\n|\\r", "\t|\n|\r', '', result) 
    #remove http link
    result = re.sub(r'http\S+', '', result)
    #remove mention
    result = re.sub(r'@\S+', '', result)
    #remove hashtags
    result = re.sub(r'#\S+', '', result)
    #remove emojis
    CLEANR = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    result = re.sub(CLEANR, '', result)
    return result.strip()


train_data['text_preprocessed']=train_data['text'].apply(lambda f : preprocess(f))
test_data['text_preprocessed']=test_data['text'].apply(lambda f : preprocess(f))
valid_data['text_preprocessed']=valid_data['text'].apply(lambda f : preprocess(f))


In [74]:
pd.set_option('display.max_columns', None)
train_data.head(50) 


,score,text,emoji,hashtags,text_preprocessed
0,0,"overgeneralized, not helpful to anyone serious...",,,"overgeneralized, not helpful to anyone serious..."
1,1,Great sound and service.,,,Great sound and service.
2,1,love this book!!!: this book is a fast read ab...,,,love this book!!!: this book is a fast read ab...
3,1,A hugely enjoyable screen version of Rona Jaff...,,,A hugely enjoyable screen version of Rona Jaff...
4,0,What an uninteresting hodge-podge. It could ha...,,,What an uninteresting hodge-podge. It could ha...
5,1,@USAirways customer service at its best! Rache...,,,customer service at its best! Rachel S. took ...
6,0,@VirginAmerica Is it normal to receive no repl...,,"baggageissues,smh",Is it normal to receive no reply from Central ...
7,0,Imagine the worst skits from Saturday Night Li...,,,Imagine the worst skits from Saturday Night Li...
8,0,This is one of the worst films ever. I like ch...,,,This is one of the worst films ever. I like ch...
9,1,@JetBlue flight attendant Wendi on Flt 127 on ...,"👍,👍",,"flight attendant Wendi on Flt 127 on 2/17, New..."


In [75]:
#usually also stop words are dropped
#nltk.download('stopwords')
#from nltk.corpus import stopwords
#stop_words = set(stopwords.words('english'))
#data['review'] = data['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
#https://www.analyticsvidhya.com/blog/2022/03/building-naive-bayes-classifier-from-scratch-to-perform-sentiment-analysis/

# Model